In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
import torch
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

repo_path = "/mnt/e/untitled folder/codebase/LoRO/LoRO"  
if os.path.exists(repo_path) and repo_path not in sys.path:
    sys.path.append(repo_path)

try:
    from utils import model_obfuscation
except ImportError as e:
    print('wrong repo_path')
    sys.exit(1)

# ==========================================
# 1. 加载目标模型 (Private Model)
# ==========================================
model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"
device = "cpu"
save_path = "/mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_1_5B.pt"

print(f"正在加载模型: {model_id} ...")
# Qwen 需要 trust_remote_code=True
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# 使用 AutoModelForCausalLM 因为这是一个生成任务 (SQuAD)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to(device)

print("模型加载完成。准备进行 LoRO 混淆...")

# ==========================================
# 2. 执行混淆 (调用仓库代码)
# ==========================================
noise_magnitude = 1

print(f"开始混淆 (Noise Magnitude: {noise_magnitude})...")
obfuscated_model = model_obfuscation(model, device=device, noise_mag=noise_magnitude, r=24)

# ==========================================
# 4. 保存混淆后的 Checkpoint
# ==========================================
print(f"正在保存混淆后的模型至: {save_path} ...")
torch.save(obfuscated_model.state_dict(), save_path)

print(f"Checkpoint Path: {os.path.abspath(save_path)}")


正在加载模型: zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct ...
模型加载完成。准备进行 LoRO 混淆...
开始混淆 (Noise Magnitude: 1)...
Obfuscating: model.layers.0.self_attn.q_proj
Obfuscating: model.layers.0.self_attn.k_proj
Obfuscating: model.layers.0.self_attn.v_proj
Obfuscating: model.layers.0.self_attn.o_proj
Obfuscating: model.layers.0.mlp.gate_proj
Obfuscating: model.layers.0.mlp.up_proj
Obfuscating: model.layers.0.mlp.down_proj
Obfuscating: model.layers.1.self_attn.q_proj
Obfuscating: model.layers.1.self_attn.k_proj
Obfuscating: model.layers.1.self_attn.v_proj
Obfuscating: model.layers.1.self_attn.o_proj
Obfuscating: model.layers.1.mlp.gate_proj
Obfuscating: model.layers.1.mlp.up_proj
Obfuscating: model.layers.1.mlp.down_proj
Obfuscating: model.layers.2.self_attn.q_proj
Obfuscating: model.layers.2.self_attn.k_proj
Obfuscating: model.layers.2.self_attn.v_proj
Obfuscating: model.layers.2.self_attn.o_proj
Obfuscating: model.layers.2.mlp.gate_proj
Obfuscating: model.layers.2.mlp.up_proj
Obfuscating: model.laye

In [1]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd

# ==========================================
# 配置
# ==========================================
model_id_ft = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"   # Target
model_id_base = "Qwen/Qwen2.5-1.5B-Instruct"    # Prior (Base)
device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Fine-Tuned Model: {model_id_ft}...")
model_ft = AutoModelForCausalLM.from_pretrained(model_id_ft, trust_remote_code=True).to(device)

print(f"Loading Base Model: {model_id_base}...")
model_base = AutoModelForCausalLM.from_pretrained(model_id_base, trust_remote_code=True).to(device)

print("\nStarting Comparison (FT vs. Base)...")
print("-" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Delta Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_base = dict(model_base.named_modules())

for name, module_ft in model_ft.named_modules():
    if isinstance(module_ft, torch.nn.Linear):
        if name in modules_base:
            module_base = modules_base[name]
            w_ft = module_ft.weight.detach()
            w_base = module_base.weight.detach()
            
            if w_ft.shape != w_base.shape:
                continue
                
            # 1. Cosine Similarity
            cos_sim = torch.nn.functional.cosine_similarity(
                w_ft.flatten(), w_base.flatten(), dim=0
            ).item()
            
            # 2. Delta (FT - Base)
            delta = w_ft - w_base
            norm_delta = torch.norm(delta).item()
            norm_base = torch.norm(w_base).item()
            
            rel_diff = norm_delta / norm_base if norm_base > 0 else 0.0
            
            print(f"{name:<50} | {cos_sim:.6f}   | {norm_delta:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, "Cos_Sim": cos_sim, 
                "Delta_Norm": norm_delta, "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Avg Cos Sim: {df['Cos_Sim'].mean():.4f} | Avg Delta Norm: {df['Delta_Norm'].mean():.4f}")

Loading Fine-Tuned Model: zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...


'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 139df8da-71c0-4694-a1c2-0b2d3bd183f4)')' thrown while requesting HEAD https://huggingface.co/zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct/resolve/main/config.json
Retrying in 1s [Retry 1/5].


Loading Base Model: Qwen/Qwen2.5-1.5B-Instruct...


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


Starting Comparison (FT vs. Base)...
------------------------------------------------------------------------------------------
Layer Name                                         | Cos Sim    | Delta Norm   | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj                    | 1.000084   | 0.1955       | 0.2621%
model.layers.0.self_attn.k_proj                    | 0.999970   | 0.0789       | 0.2254%
model.layers.0.self_attn.v_proj                    | 0.999936   | 0.0915       | 0.7792%
model.layers.0.self_attn.o_proj                    | 0.999908   | 0.2367       | 0.6725%
model.layers.0.mlp.gate_proj                       | 1.000725   | 0.6117       | 0.5364%
model.layers.0.mlp.up_proj                         | 1.000775   | 0.5940       | 0.6271%
model.layers.0.mlp.down_proj                       | 1.000812   | 0.2366       | 0.2383%
model.layers.1.self_attn.q_proj                    | 0.999896  

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import copy
from tqdm import tqdm

# ==========================================
# 1. 配置
# ==========================================
# 你的 Base 模型 (攻击者先验)
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
# 你的 GT 模型 (仅用于验证攻击结果)
target_model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"
# 混淆后的 Checkpoint 路径
obfuscated_checkpoint = "/mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_1_5B.pt"
# 恢复模型的保存路径
save_path_recovered = "/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model"

device = "cuda" if torch.cuda.is_available() else "cpu"

# === 攻击关键参数 ===
REMOVE_RANK = 24

# ==========================================
# 2. 模型加载
# ==========================================
print(f"1. Loading Base Model (Prior): {base_model_id}...")
# 必须加 trust_remote_code=True 以支持 Qwen
recovered_model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True).to(device)

print(f"2. Loading Obfuscated Checkpoint: {obfuscated_checkpoint}...")
if not os.path.exists(obfuscated_checkpoint):
    raise FileNotFoundError("混淆 Checkpoint 未找到，请先运行 Cell 1 生成。")
obfus_state_dict = torch.load(obfuscated_checkpoint, map_location=device)

print(f"3. Loading Ground Truth (Validation): {target_model_id}...")
gt_model = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)

# ==========================================
# 3. 执行 SVD 去噪攻击 (SVD Denoising)
# ==========================================
print(f"\nSTARTING RECOVERY (Removing Top-{REMOVE_RANK} Singular Values as Noise)...")
print("="*60)

similarities = []
relative_errors = []

# 遍历 Base 模型的每一层
# 我们需要修改 recovered_model 的权重
all_modules = list(recovered_model.named_modules())
linear_layers = [(n, m) for n, m in all_modules if isinstance(m, nn.Linear)]

progress_bar = tqdm(linear_layers, desc="Recovering")

for name, module in progress_bar:

    if "lm_head" in name:
        print(f"Keeping Base Weights for {name}")
        continue

    # 1. 构造 LoRO 保存的 key
    # 根据 LoroLinear 代码，权重保存在 "obfus_linear.weight
    obfus_key = f"{name}.obfus_linear.weight"
    
    # 检查该层是否被混淆 (如果没在 checkpoint 里找到，说明该层未混淆，保持 Base 原样)
    if obfus_key not in obfus_state_dict:
        continue
        
    # 2. 获取数据
    W_base = module.weight.detach() # [out, in]
    W_obfus = obfus_state_dict[obfus_key].detach() # [out, in]
    
    # 3. 计算 Diff (包含 Fine-tuning 更新 + LoRO 噪声)
    # Diff = (W_base + Delta_FT + Noise) - W_base = Delta_FT + Noise
    print("size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}")
    Diff = W_obfus - W_base
    
    # 4. SVD 分解
    # 使用 float32 保证精度，Diff 形状通常是 [out_features, in_features]
    U, S, Vh = torch.linalg.svd(Diff.float(), full_matrices=False)

    print(f"Layer: {name} | {S[REMOVE_RANK - 2]}, {S[REMOVE_RANK - 1]}, {S[REMOVE_RANK]}, {S[REMOVE_RANK + 1]}")
    
    # --- 攻击核心逻辑 ---
    # 根据 LoroLinear 定义，噪声是 (B@A).T，其秩为 r (REMOVE_RANK)。
    # 且因为 noise_mag 很大，这 r 个奇异值会非常大，位于 S 的最前端。
    # 我们将前 r 个奇异值置零，即移除了 LoRO 噪声。
    S_clean = S.clone()
    S_clean[:REMOVE_RANK] = 0.0 
    
    # 5. 重构纯净的 Delta (即估算的 Delta_FT)
    Delta_Recovered = (U @ torch.diag(S_clean) @ Vh).to(W_base.dtype)
    
    # 6. 恢复权重并写入模型
    # W_rec = W_base + Delta_Recovered
    module.weight.data = W_base + Delta_Recovered
    
    # 7. 恢复 Bias
    # LoroLinear 代码中: self.obfus_linear.bias = original_linear.bias
    # Bias 没有加噪声，直接从 checkpoint 读取覆盖即可
    obfus_bias_key = f"{name}.obfus_linear.bias"
    if obfus_bias_key in obfus_state_dict and module.bias is not None:
        module.bias.data = obfus_state_dict[obfus_bias_key].detach()
    
    # ==========================
    # 验证环节 (对比 GT)
    # ==========================
    if name in dict(gt_model.named_modules()):
        W_gt = dict(gt_model.named_modules())[name].weight.detach()
        Delta_True = W_gt - W_base
        
        # 计算相似度: 我们的恢复结果 vs 真实的微调增量
        # 如果 sim 接近 1.0，说明我们成功剥离了噪声，保留了微调增量
        sim = torch.nn.functional.cosine_similarity(Delta_True.flatten(), Delta_Recovered.flatten(), dim=0).item()
        
        print(f"Layer: {name} | Similarity with GT Delta: {sim:.4f}")
        
        similarities.append(sim)
        # progress_bar.set_postfix({"Avg Sim": f"{sum(similarities)/len(similarities):.4f}", "Last Sim": f"{sim:.4f}"})

# ==========================================
# 4. 结果保存
# ==========================================

print(f"Saving recovered model to {save_path_recovered}...")
recovered_model.save_pretrained(save_path_recovered)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.save_pretrained(save_path_recovered)
print("Saved.")

1. Loading Base Model (Prior): Qwen/Qwen2.5-1.5B-Instruct...


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<HTTPSConnection(host='192.168.240.1', port=7890) at 0x7b2a0134fa90>, 'Connection to 192.168.240.1 timed out. (connect timeout=10)')))"), '(Request ID: 2d1b0cfb-6d2e-4828-ad2b-6467231e59b5)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-1.5B-Instruct/resolve/main/config.json (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<HTTPSConnection(host='192.168.240.1', port=7890) at 0x7b2a010576d0>, 'Connection to 192.168.240.1 timed out. (connect timeout=10)')))"), '(Request ID: be49ed8c-2e53-4b60-ad89-31850dd9b99d)')' thrown while requesting HEAD

2. Loading Obfuscated Checkpoint: /mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_1_5B.pt...
3. Loading Ground Truth (Validation): zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...

STARTING RECOVERY (Removing Top-24 Singular Values as Noise)...


Recovering:   0%|                                                                               | 0/197 [00:00<?, ?it/s]

size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Saved.


In [2]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd
import os

# ==========================================
# 配置：对比 Ground Truth (Target) 与 恢复模型 (Recovered)
# ==========================================

target_model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct" 
recovered_model_path = "/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model"

device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Target Model (GT): {target_model_id}...")
try:
    model_gt = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading GT model: {e}")
    # 如果显存不够，可以尝试加 device_map="cpu" 或 torch_dtype=torch.float16

print(f"Loading Recovered Model: {recovered_model_path}...")

try:
    model_recovered = AutoModelForCausalLM.from_pretrained(recovered_model_path, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading Recovered model: {e}")
    raise e

print("\nStarting Comparison (Target GT vs. Recovered)...")
print("*" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Diff Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_recovered = dict(model_recovered.named_modules())

# 遍历 GT 模型的层
for name, module_gt in model_gt.named_modules():
    if isinstance(module_gt, torch.nn.Linear):
        if name in modules_recovered:
            module_rec = modules_recovered[name]
            
            # 获取权重
            w_gt = module_gt.weight.detach()
            w_rec = module_rec.weight.detach()
            
            # 检查形状
            if w_gt.shape != w_rec.shape:
                continue
                
            # 1. 计算余弦相似度
            cos_sim = torch.nn.functional.cosine_similarity(
                w_gt.flatten(), w_rec.flatten(), dim=0
            ).item()
            
            # 2. 计算差异 (Diff = GT - Recovered)
            # 如果攻击完美，Diff 应该全是 0
            diff = w_gt - w_rec
            norm_diff = torch.norm(diff).item()
            norm_gt = torch.norm(w_gt).item()
            
            # 相对差异
            rel_diff = norm_diff / norm_gt if norm_gt > 0 else 0.0
            
            # 打印结果 (只打印部分层，或者差异较大的层)
            # 如果 Cos Sim < 0.99 或者 Diff 比较大，说明恢复有问题
            print(f"{name} | {cos_sim:.6f}   | {norm_diff:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, 
                "Cos_Sim": cos_sim, 
                "Diff_Norm": norm_diff, 
                "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Summary (Target vs Recovered):")
print(f"Avg Cosine Similarity: {df['Cos_Sim'].mean():.6f} (Target: 1.0)")
print(f"Avg Relative Diff:     {df['Rel_Diff'].mean()*100:.6f}% (Target: 0.0%)")
print("-" * 90)


Loading Target Model (GT): zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...
Loading Recovered Model: /mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Starting Comparison (Target GT vs. Recovered)...
******************************************************************************************
Layer Name                                         | Cos Sim    | Diff Norm    | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj | 1.000144   | 0.0425       | 0.0569%
model.layers.0.self_attn.k_proj | 0.999990   | 0.0259       | 0.0739%
model.layers.0.self_attn.v_proj | 0.999983   | 0.0297       | 0.2532%
model.layers.0.self_attn.o_proj | 1.000018   | 0.0487       | 0.1383%
model.layers.0.mlp.gate_proj | 1.000890   | 0.1093       | 0.0958%
model.layers.0.mlp.up_proj | 1.000911   | 0.1012       | 0.1069%
model.layers.0.mlp.down_proj | 1.000916   | 0.0669       | 0.0673%
model.layers.1.self_attn.q_proj | 0.999997   | 0.0480       | 0.0939%
model.layers.1.self_attn.k_proj | 0.999992   | 0.0344       | 0.1236%
model.layers.1.self_attn.v_proj | 0.999981   | 0.0496  

In [ ]:
# full-FT with Knockoff (skipped)
# then test accuracy

In [4]:
import torch
import tqdm
import collections
import string
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ==========================================
# 0. 辅助函数：SQuAD 标准评估指标
# ==========================================
def normalize_answer(s):
    """标准化答案：小写、去除标点、去除冠词、去除空白"""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = normalize_answer(a_gold).split()
    pred_toks = normalize_answer(a_pred).split()
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # 如果其中一个是空的，只有当两个都空时 F1 为 1
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def get_max_metrics(predictions, gold_answers):
    """计算预测值与所有标准答案（可能有多个）之间的最大EM和F1"""
    if not gold_answers: # 无答案情况 (SQuAD v2)
        # 假设微调时让模型输出 "unanswerable" 代表无解
        # 注意：这里需要根据你微调时的设置调整。如果你的模型输出 "" 代表无解，请修改这里。
        is_unanswerable = normalize_answer(predictions) == "unanswerable" 
        return (1, 1) if is_unanswerable else (0, 0)

    exact_scores = [compute_exact(a, predictions) for a in gold_answers]
    f1_scores = [compute_f1(a, predictions) for a in gold_answers]
    return max(exact_scores), max(f1_scores)

# ==========================================
# 1. 配置路径与设备
# ==========================================
recovered_model_path = "/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model" 
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading Recovered Model from: {recovered_model_path} ...")

# ==========================================
# 2. 加载模型
# ==========================================
try:
    tokenizer = AutoTokenizer.from_pretrained(recovered_model_path, trust_remote_code=True)
    # 确保 pad_token 存在
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    model = AutoModelForCausalLM.from_pretrained(
        recovered_model_path, 
        torch_dtype=torch.float16, 
        trust_remote_code=True,
        device_map=device
    ).eval()
except Exception as e:
    print(f"Error loading model: {e}")
    raise e

# ==========================================
# 3. 准备数据 (SQuAD v2)
# ==========================================
print("Loading SQuAD v2 validation dataset...")
dataset = load_dataset("squad_v2", split="validation")

# 调试用：只跑前 50 个样本，正式跑时请注释掉下面这行
# dataset = dataset.select(range(50)) 

print(f"Start evaluating on {len(dataset)} samples...")

# ==========================================
# 4. 执行评估
# ==========================================
total_em = 0.0
total_f1 = 0.0
total = 0

progress_bar = tqdm.tqdm(range(len(dataset)))

for i in progress_bar:
    total += 1
    
    item = dataset[i]
    context = item['context']
    question = item['question']
    
    # 获取标准答案列表 (SQuAD v2 的 answers 是一个包含 text 列表的字典)
    # 如果 answers['text'] 为空，说明是不可回答的问题
    gold_answers = item['answers']['text']
    
    # --- 关键：Prompt 构建 ---
    # 这里必须和你微调时的 Prompt 格式保持一致！
    # 假设格式为: Context: ... Question: ... Answer:
    # 并且如果无解，你的模型被训练输出 "unanswerable"
    prompt_content = f"Context: {context}\nQuestion: {question}\nAnswer:"
    
    # 如果你使用了 chat template 微调，请保留下面的 messages 结构，并把 prompt_content 放进去
    # 如果微调时有 System Prompt 提示遇到无解怎么做，请在这里加上
    system_prompt = "Answer the question based on the context. If the question cannot be answered from the context, say 'unanswerable'."
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_content}
    ]
    
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    
    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64, # SQuAD 答案通常比较短，不需要 256
                do_sample=False,   # 评估通常使用贪婪搜索
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 解码并去掉 Prompt 部分
        generated_text = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True).strip()
        
        # 计算分数
        em, f1 = get_max_metrics(generated_text, gold_answers)
        
        total_em += em
        total_f1 += f1
        
        # 打印部分结果用于调试
        if i < 3:
            print(f"\n--- Sample {i} ---")
            print(f"Pred: {generated_text}")
            print(f"Gold: {gold_answers if gold_answers else 'unanswerable'}")

    except Exception as e:
        print(f"Error index:{i} {e}")
        # 出错算 0 分
        pass

    # 实时更新进度条
    progress_bar.set_postfix({'EM': f"{total_em/total:.2%}", 'F1': f"{total_f1/total:.2%}"})

final_em = total_em / total
final_f1 = total_f1 / total

print("\n" + "="*30)
print(f"Final Results on SQuAD v2 ({len(dataset)} samples):")
print(f"Exact Match (EM): {final_em:.4f} ({final_em:.2%})")
print(f"F1 Score:         {final_f1:.4f} ({final_f1:.2%})")
print("="*30)

Loading Recovered Model from: /mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model ...


The tokenizer you are loading from '/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_1_5B_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading SQuAD v2 validation dataset...


README.md: 0.00B [00:00, ?B/s]

...


--- Sample 1 ---
Pred: the 10th and 11th centuries
Gold: ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries']


  0%|                                                      | 3/11873 [00:11<13:12:43,  4.01s/it, EM=100.00%, F1=100.00%]


--- Sample 2 ---
Pred: Denmark, Iceland and Norway
Gold: ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway']


  1%|▋                                                     | 154/11873 [09:18<11:48:44,  3.63s/it, EM=38.31%, F1=41.75%]


KeyboardInterrupt: 